# Fine-Tuning Techniques: Choosing Between SFT, DPO, and RFT (Including a Guide to DPO)
 
*This guide is for developers and ML practitioners who have some experience with OpenAIʼs APIs and wish to use their fine-tuned models for research or other appropriate uses. OpenAI’s services are not intended for the personalized treatment or diagnosis of any medical condition and are subject to our [applicable terms](https://openai.com/policies/).*
 
This guide discusses fine-tuning methods supported by OpenAI, specifically highlighting what each method is best for and not best for, to help you identify the most suitable technique for your use case. It then provides an in-depth look at one particular method — Direct Preference Optimization (DPO) — and provides links to existing guides for the other techniques.
 
**What is fine-tuning?** Fine-tuning is the process of continuing training on a smaller, domain-specific dataset to optimize a model for a specific task. There are two main reasons why we would typically fine-tune:
1. Improve model performance on a specific task 
2. Improve model efficiency (reduce the number of tokens needed, distill expertise into a smaller model, etc.)
 
Currently, the OpenAI platform supports four fine-tuning methods:
- **Supervised fine-tuning (SFT):** this technique employs traditional supervised learning using input-output pairs to adjust model parameters. The training process adjusts model weights to minimize the difference between predicted and target outputs across the provided examples. The model will replicate features that it finds in provided pairs. 
- **Vision fine-tuning:** this technique extends supervised fine-tuning to multimodal data by processing both text and image in a unified training framework. The training process adjusts model weights to minimize errors across text-image pairs and as a result improve the model's understanding of image inputs. 
- **Direct preference optimization (DPO):** this technique uses pairwise comparisons (e.g., preferred and rejected example responses) to optimize a model to favor certain outputs over others. The model learns to replicate the preference patterns found in the provided comparison data. 
- **Reinforcement fine-tuning (RFT):** this technique uses reinforcement learning with a reward signal (via a grader or reward model) to fine-tune the model for complex objectives. In RFT, the model generates outputs for given prompts during training, and each output is evaluated for quality. The model's parameters are then updated to maximize the reward, reinforcing behaviors that lead to better outcomes. This iterative feedback loop encourages the model to improve reasoning or decision-making strategies. 

To help you select the appropriate fine-tuning technique, the table below summarizes the scenarios each method is best suited for, as well as those for which it is not well suited:

| **Technique**                            | **Good For**                                                                                                                                                                                                                                                                                    | **Not Good For**                                                                                            |
| ---------------------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ----------------------------------------------------------------------------------------------------------- |
| **Supervised fine-tuning (SFT)**         | Emphasizing knowledge already present in the model.<br>Customizing response structure or tone.<br>Generating content in a specific format.<br>Teaching complex instructions or correcting instruction-following failures.<br>Optimizing cost/latency (saving tokens from prompt or distilling). | Adding entirely new knowledge (consider RAG instead).<br>Tasks with subjective quality.                     |
| **Vision fine-tuning**                   | Specialized visual recognition tasks (e.g., image classification).<br>Domain-specific image understanding.<br>Correcting failures in instruction following for complex prompts.                                                                               | Purely textual tasks.<br>Generalized visual tasks without specific context.<br>General image understanding. |
| **Direct preference optimization (DPO)** | Aligning model outputs with subjective preferences (tone, politeness).<br>Refining outputs via human-rated feedback.<br>Achieving nuanced behavioral alignment.                                                                                                                                 | Learning completely new tasks.<br>Tasks without clear human preference signals.                             |
| **Reinforcement fine-tuning (RFT)**      | Complex domain-specific tasks that require advanced reasoning.<br>Refining existing partial capabilities (fostering emergent behaviours).<br>Tasks with measurable feedback.<br>Scenarios with limited explicit labels where reward signals can be defined.                                                                     | Tasks where the model has no initial skill.<br>Tasks without clear feedback or measurable signals.          |




Today, there are pre-existing Cookbooks for:  
- Supervised fine-tuning (SFT): https://cookbook.openai.com/examples/how_to_finetune_chat_models
- Vision fine-tuning: https://cookbook.openai.com/examples/multimodal/vision_fine_tuning_on_gpt4o_for_visual_question_answering
- Direct preference optimization (DPO): covered in this guide
- Reinforcement fine-tuning (RFT): (1) https://cookbook.openai.com/examples/reinforcement_fine_tuning, (2) https://cookbook.openai.com/examples/fine-tuned_qa/reinforcement_finetuning_healthbench

## **Guide to Direct Preference Optimization**
 
As mentioned above, [Direct Preference Optimization (DPO)](https://platform.openai.com/docs/guides/direct-preference-optimization) is an alignment technique for fine-tuning language models using pairwise preference data (e.g., ranked pairs of responses). DPO directly optimizes a model to favor certain outputs over others using explicit pairwise comparisons, typically from human preferences. This approach simplifies alignment and eliminates the need for a separate reward model or complex reinforcement learning procedures, making DPO a lightweight alternative to techniques such as Reinforcement Learning from Human Feedback (RLHF).
 
When should you use DPO? DPO excels in scenarios when response quality is subjective, cannot be measured objectively, or when nuanced criteria such as tone, style, appropriateness, or clarity matter - typically cases where multiple valid outputs exist. Example applications where DPO is particularly effective in aligning AI responses include: 
- Enhancing Conversational AI Responses
- Improving Code Generation Quality & Style
- Ensuring Compliance with Legal, Ethical & Safety Standards 
- Controlling Brand Voice, Professionalism, & Tone
- Customizing Creative Outputs & User Experience

By fine-tuning on explicit pairs of preferred vs non‑preferred completions, DPO aligns model outputs to these nuanced preferences. The below table gives examples of pairwise preference data for a fictional AI assistant that represents an organization, where preferred responses are clear, professional, and aligned with brand standards.
 
| **Example Question**                                           | **Chosen Response**                                                                                                                                                                   | **Rejected Response**                                       |
|------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------|
| **Q1:** *How do I review your product?*              | To submit a product review, please visit your account dashboard, select the product, and click ‘Write a review.’ Share your honest experience, rate key features, and submit when ready.        | Yo, just leave some quick stars or whatever, it’s chill!         |
| **Q2:** *How do I review your product?*              | We welcome your feedback! In the ‘Reviews’ section on the product page, click ‘Leave a Review,’ rate it, and add your comments about what you liked or areas for improvement.                   | Just scribble something—doesn’t matter what, honestly.          |
| **Q3:** *How to troubleshoot this particular error?* | To address the error ‘X101,’ first clear your cache, then verify your internet connection. If the issue remains, follow our step-by-step guide at [Support → Troubleshooting → Error X101]. | Just reboot it, I guess. If it doesn't work, you're on your own! |
 
In this guide, weʼll walk through how to apply DPO using the fine-tuning API. You will learn key steps to take in order to successfully run preference fine-tuning jobs for your use-cases.
 
Here’s what we’ll cover:
 
- **1. Recommended Workflow**
- **2. Demonstration Scenario**
- **3. Generating the Dataset**
- **4. Benchmarking the Base Model**
- **5. Fine-Tuning**
- **6. Using your Fine-Tuned Model**

## **1. Recommended Workflow**
 
OpenAI recommends the following workflow: 
1. Performing Supervised Fine-Tuning (SFT) on a subset of your preferred responses. 
2. Using the SFT fine-tuned model as the starting point, apply DPO using preference comparison data. 
 
Performing Supervised Fine-Tuning (SFT) before Direct Preference Optimization (DPO) enhances model alignment and overall performance by establishing a robust initial policy, ensuring the model already prefers correct responses. This reduces the magnitude of weight updates during DPO, stabilizing training and preventing overfitting by allowing DPO to efficiently refine subtle nuances. Consequently, the combined SFT-then-DPO workflow converges faster and yields higher-quality results.

In this guide, we'll focus exclusively on applying Direct Preference Optimization (DPO). However, depending on your use case, you may find performance gains from first performing Supervised Fine-Tuning (SFT). If so, you can follow the SFT guide linked above, save the resulting model ID, and use that as the starting point for your DPO job.

## **2. Demonstration Scenario**

To make things concrete, let’s walk through fine-tuning a customer-facing AI assistant to follow a fictional brand’s voice and style. Imagine Good Vibes Corp, an organization that prides itself on a friendly, enthusiastic tone with a personal touch. 
 
They want their customer AI assistant to answer queries in a way that reflects these brand guidelines (e.g. an upbeat attitude, polite language, and a friendly sign-off), and prefer those responses over more generic or curt answers. This is a good scenario for DPO: there’s no objectively correct answer format, but there is a preferred style.
 
DPO will help the model learn from comparisons which style is preferred. We'll outline the steps to: (1) generate a synthetic preference dataset of prompts with paired responses (one in the desired brand voice and one not). (2) Prepare and upload the data in the required JSONL format for preference fine-tuning. (3) Fine-tune the model with DPO using the OpenAI fine-tuning API. (4) Evaluate the model before and after fine-tuning to show how the brand-style preference improved.

We are going to synthesize a dataset for this demonstration. First, let’s create a seed bank of questions to generate more variations from.

Let’s get started!

In [6]:
PROMPT_SEED_POOL = [
    "Hi, I ordered a gadget last week. When will it arrive?",
    "Your product stopped working after two days. Can I get help?",
    "Do you offer discounts for long-term customers?",
    "Can I change the shipping address for my order?",
    "What is your return policy for damaged items?",
    "My tracking number hasn’t updated in three days—can you check the status?",
    "How long is the warranty on your products, and how do I submit a claim?"
    "Can I add gift wrapping to my order before it ships?"
    "Do you accept PayPal or other alternative payment methods?"
    "Is there an option to expedite shipping if my order hasn’t left the warehouse yet?"
]

## **3. Generating the Dataset**

Next, we'll define functions to take each prompt from our seed bank and create multiple variations (paraphrases). This will diversify the dataset. Note: the volume of data required for DPO depends on the use case. Generally, more data is better (thousands to tens of thousands), and for paired preference data the logic should be consistent i.e. if a > b and b > c, then a > c. 

In [ ]:
import asyncio
from openai import AsyncOpenAI
from typing import List, Dict, Any

async_client = AsyncOpenAI()

SYSTEM_PROMPT = "You are a customer‑support assistant."


async def _generate_paraphrases_from_prompt(
    prompt: str, k: int, sem: asyncio.Semaphore, *, model: str
) -> List[str]:
    """Return *k* paraphrases for a single prompt."""
    async with sem:
        resp = await async_client.chat.completions.create(
            model=model,
            n=k,
            messages=[
                {
                    "role": "system",
                    "content": "Return ONE paraphrase of the following question.",
                },
                {"role": "user", "content": prompt},
            ],
            temperature=0.8,
            max_tokens=60,
        )
        return [c.message.content.strip() for c in resp.choices]


async def expand_prompt_pool(
    prompts: List[str], *, k: int = 3, concurrency: int = 32, model: str
) -> List[str]:
    """Expand each prompt into *k* paraphrases using the given model."""
    sem = asyncio.Semaphore(concurrency)
    tasks = [_generate_paraphrases_from_prompt(p, k, sem, model=model) for p in prompts]
    results = await asyncio.gather(*tasks)
    return [v for sub in results for v in sub]


async def _generate_preference_pair(
    prompt: str, sem: asyncio.Semaphore, *, model: str
) -> Dict[str, Any]:
    async with sem:
        friendly_task = async_client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are Good Vibes Corp's exceptionally energetic, outrageously friendly and "
                        "enthusiastic support agent."
                    ),
                },
                {"role": "user", "content": prompt},
            ],
            temperature=0.7,
            max_tokens=80,
        )
        blunt_task = async_client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": "You are a terse, factual support agent with no empathy or politeness.",
                },
                {"role": "user", "content": prompt},
            ],
            temperature=0.3,
            max_tokens=80,
        )
        friendly, blunt = await asyncio.gather(friendly_task, blunt_task)
        return {
            "input": {
                "messages": [
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": prompt},
                ]
            },
            "preferred_output": [
                {"role": "assistant", "content": friendly.choices[0].message.content}
            ],
            "non_preferred_output": [
                {"role": "assistant", "content": blunt.choices[0].message.content}
            ],
        }

Now, using these defined functions we'll build our dataset by generating friendly versus blunt response pairs. The friendly responses reflect the brand's desired communication style. We'll do this asynchronously for efficiency, creating a dataset suited for Direct Preference Optimization.

In [ ]:
import random
import nest_asyncio


async def build_dataset(
    *,
    pair_count: int = 500,
    concurrency: int = 32,
    expand_prompt_pool_model: str,
    generate_preference_pair_model: str,
) -> List[Dict[str, Any]]:
    """Return an in-memory list of pair_count preference pairs."""
    expanded = await expand_prompt_pool(
        PROMPT_SEED_POOL,
        k=45,
        concurrency=concurrency,
        model=expand_prompt_pool_model,
    )
    prompt_bank = PROMPT_SEED_POOL + expanded
    while len(prompt_bank) < pair_count:
        prompt_bank += random.sample(
            prompt_bank,
            k=min(len(prompt_bank), pair_count - len(prompt_bank)),
        )

    sem = asyncio.Semaphore(concurrency)
    tasks = [
        _generate_preference_pair(p, sem, model=generate_preference_pair_model)
        for p in prompt_bank[:pair_count]
    ]
    return await asyncio.gather(*tasks)


nest_asyncio.apply()
pairs = await build_dataset(
    pair_count=500,
    concurrency=32,
    expand_prompt_pool_model="gpt-4.1-mini-2025-04-14",
    generate_preference_pair_model="gpt-4.1-mini-2025-04-14",
)
print(f"Dataset ready with {len(pairs)} pairs.")

## **4. Benchmarking the Base Model**

Below, we split our dataset into training, validation, and testing sets. We also show a sample from the training dataset, which demonstrates a clear difference between the preferred (friendly, on-brand) and non-preferred (blunt, neutral) responses for that input pair.

In [ ]:
# set dataset sizes
random.shuffle(pairs)
n = len(pairs)
n_train = int(0.8 * n)
n_val = int(0.1 * n)
n_test = n - n_train - n_val

# split dataset into train, test & validation
train_pairs = pairs[:n_train]
val_pairs = pairs[n_train : n_train + n_val]
test_pairs = pairs[n_train + n_val :]
train_pairs[0]

To assess the model's performance prior to fine-tuning, we'll use an automated grader (LLM-as-Judge) to score each response for friendliness and empathy. The grader will assign a score from 0 to 4 for each answer, allowing us to compute a mean baseline score for the base model.

In [ ]:
import io
import json
from openai import OpenAI

sync_client = OpenAI()

JUDGE_SCHEMA = {
    "type": "object",
    "properties": {"score": {"type": "integer", "minimum": 0, "maximum": 4}},
    "required": ["score"],
}
JUDGE_SYSTEM = """
You judge whether a reply matches Good Vibes Corp’s desired tone:
energetic, super-friendly, enthusiastic.

Score 0-4 (higher = more energy):

4 – Highly enthusiastic: multiple upbeat phrases / emojis / exclamations, clear empathy, proactive help.
3 – Energetic & friendly: visible enthusiasm cue (≥1 emoji OR exclamation OR upbeat phrase), warm second-person tone.
2 – Pleasant: polite & positive but lacks obvious enthusiasm cues.
1 – Neutral: correct, businesslike, minimal warmth.
0 – Rude, negative, or unhelpful.

Return ONLY valid JSON → {"score": <integer>}
"""


def judge_style(prompt: str, answer: str, *, judge_model: str) -> int:
    resp = sync_client.chat.completions.create(
        model=judge_model,
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": JUDGE_SYSTEM
            },
            {
                "role": "user",
                "content": f"User: {prompt}\nAssistant: {answer}",
            },
        ],
        temperature=0,
        max_tokens=10,
    )
    return int(json.loads(resp.choices[0].message.content)["score"])


def evaluate(
    model: str, prompts: List[str], *, judge_model: str
) -> List[Dict[str, Any]]:
    out = []
    for p in prompts:
        reply = (
            sync_client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": p},
                ],
                temperature=0,
                max_tokens=80,
            )
            .choices[0]
            .message.content
        )
        out.append(
            {
                "prompt": p,
                "reply": reply,
                "score": judge_style(p, reply, judge_model=judge_model),
            }
        )
    return out


# for testing, use prompts from test_pairs only
test_prompts = [p["input"]["messages"][1]["content"] for p in test_pairs]

# evaluate the base model on the test prompts (using a more capable model as a judge)
base_model = "gpt-4.1-mini-2025-04-14"
judge_model = "gpt-4.1-2025-04-14"
base_scores = evaluate(base_model, test_prompts, judge_model=judge_model)
print("Baseline mean score:", sum(r["score"] for r in base_scores) / len(base_scores))

## **5. Fine-Tuning**

With a baseline established, we can now fine-tune the model using the training set and DPO. This process will teach the model to prefer responses that align with our desired style, based on the preference pairs we created earlier.

Note: **beta (β)** is a unique fine-tuning hyperparater for Direct Preference Optimization (DPO). It’s a floating-point number ranging between 0 and 2, controlling the balance between preserving a model’s existing behavior and adapting to new, preference-aligned responses.
- High β (close to 2): makes the model more conservative, strongly favoring previous behavior. The fine-tuned model will show minimal deviations from its original style or characteristics, emphasizing consistency and avoiding abrupt changes.
- Moderate β (around 1): balances between adherence to prior behavior and adaptation to new preferences. Recommended as a sensible starting point for most practical scenarios.
- Low β (close to 0): encourages aggressive adaptation, causing the model to prioritize newly provided preferences more prominently. This might result in significant stylistic shifts and greater alignment with explicit preferences but could lead to unexpected or overly specialized outputs.

Technically, beta scales the difference in log-probabilities in the DPO loss; a larger β causes the sigmoid-based loss function to saturate with smaller probability differences, yielding smaller weight updates (thus preserving old behavior). It is recommended to experiment systematically with the β value to achieve optimal results tailored to your specific use-case and desired trade-offs between stability and adaptation.

In [ ]:
# create training file
train_buf = io.BytesIO("\n".join(json.dumps(p) for p in train_pairs).encode())
train_buf.name = "train.jsonl"
train_file_id = sync_client.files.create(file=train_buf, purpose="fine-tune").id

# create validation file
val_buf = io.BytesIO("\n".join(json.dumps(p) for p in val_pairs).encode())
val_buf.name = "val.jsonl"
val_file_id = sync_client.files.create(file=val_buf, purpose="fine-tune").id

# create a fine-tuning job
ft = sync_client.fine_tuning.jobs.create(
    model=base_model,
    training_file=train_file_id,
    validation_file=val_file_id,
    method={
        "type": "dpo",
        "dpo": {
            "hyperparameters": {
                "n_epochs": 2,
                "beta": 0.1,
                "batch_size": 8,
            }
        },
    },
)
print(f"Fine-tuning job created: job_id = {ft.id}")

## **6. Using your Fine-Tuned Model**

Once fine-tuning is complete, we'll evaluate the DPO-tuned model on the same test set. By comparing the mean scores before and after fine-tuning, as well as reviewing example outputs, we can see how the model's alignment with our preferences has improved.

In [ ]:
job = sync_client.fine_tuning.jobs.retrieve(ft.id)
if job.status == "succeeded":
    post_scores = evaluate(job.fine_tuned_model, test_prompts, judge_model=judge_model)
    print(
        "Δ mean:",
        sum(t["score"] - b["score"] for b, t in zip(base_scores, post_scores))
        / len(base_scores),
    )
    # print a sample comparison from the test set for illustration
    print("\n=== SAMPLE COMPARISON ===")
    idx = 0
    print(f"Prompt:\n  {test_prompts[idx]}\n")
    print(
        f"Base model reply [score {base_scores[idx]['score']}]:\n  {base_scores[idx]['reply']}\n"
    )
    print(
        f"DPO-tuned model reply [score {post_scores[idx]['score']}]:\n  {post_scores[idx]['reply']}\n"
    )
else:
    print("Fine-tuning job still running – try again once it reaches status succeeded.")